<a href="https://colab.research.google.com/github/PB811/Document-VQA/blob/main/Gemma3_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version

Python 3.11.12


In [ ]:
!pip install ollama pillow streamlit pyngrok Pillow pdf2image "numpy<2"

In [ ]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.8 [186 kB]
Fetched 186 kB in 1s (317 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 126102 

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!ollama --version

ollama version is 0.6.8


In [ ]:
import requests
response = requests.get('http://localhost:11434')
print(response.status_code, response.text)

200 Ollama is running


In [ ]:
# Start the Ollama service in the background
!ollama serve > /dev/null 2>&1 &

# Wait for Ollama to initialize
import time
print("Starting Ollama service...")
time.sleep(5)
print("Ollama service should be running now")

Starting Ollama service...
Ollama service should be running now


In [ ]:
# First, make sure ollama is running
!pkill ollama  # Stop any existing instances
!ollama serve > /dev/null 2>&1 &
import time
print("Starting Ollama service...")
time.sleep(5)

# Check if NVIDIA GPU is available in the Colab environment
!nvidia-smi

# Check if Ollama can detect the GPU
!curl -s http://localhost:11434/api/tags | grep -i "gpu"

# Another way to check if Ollama is using GPU
# Pull a model if you haven't already
# !ollama pull llama2

# Run a simple inference and check for GPU usage
!nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv,noheader

# Check if the ollama process is using CUDA
!ps aux | grep ollama | grep -v grep

Starting Ollama service...
Mon May 12 08:33:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+--------------------

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use the first GPU
!ollama serve > /dev/null 2>&1 &

In [ ]:
!ollama run gemma3:12b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling e8ad13eff07a:   0% ▕▏  977 B/8.1 GB                  pulling manifest 
pulling e8ad13eff07a:   1% ▕▏  75 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a:   2% ▕▏ 142 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a:   2% ▕▏ 168 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a:   3% ▕▏ 218 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a:   3% ▕▏ 264 MB/8.1 GB                  pulling manifest 
pulling e8ad13eff07a:   4% ▕▏ 288 MB/8.1 GB         

In [ ]:
# This script sets up a Streamlit app with Ollama and Gemma-3 for OCR in Google Colab

# Setup environment
import streamlit as st
import ollama
from PIL import Image
import io
import base64
import os
import subprocess
import time
import json
from pyngrok import ngrok


In [ ]:
# Create a state file to share information between processes
STATE_FILE = '/content/app_state.json'

In [ ]:
# Ensure Ollama is running
def ensure_ollama_running():
    # Check if Ollama is running
    status = subprocess.run("pgrep ollama", shell=True, capture_output=True)
    if status.returncode != 0:
        print("Starting Ollama service...")
        subprocess.run("ollama serve > /dev/null 2>&1 &", shell=True)
        time.sleep(5)
    print("Ollama service should be running")

In [ ]:
def create_gemma_logo():
    # Create a simple placeholder logo if the original isn't available
    from PIL import Image, ImageDraw
    img = Image.new('RGB', (200, 200), color = (73, 109, 137))
    d = ImageDraw.Draw(img)
    d.text((50, 100), "Gemma-3", fill=(255, 255, 0))
    img.save('/content/gemma3.png')
    print("Created placeholder logo at /content/gemma3.png")

In [ ]:
# Check if gemma3:12b model is available and pull it if not
def ensure_gemma3_model():
    print("Checking for gemma3:12b model...")
    try:
        models = subprocess.run("ollama list", shell=True, capture_output=True, text=True)
        if "gemma3:12b" not in models.stdout:
            print("gemma3:12b model not found. Pulling it now...")
            print("This may take a few minutes depending on your internet speed...")
            pull_result = subprocess.run("ollama pull gemma3:12b", shell=True, capture_output=True, text=True)
            if "successfully pulled" in pull_result.stdout.lower():
                print("✅ Successfully pulled gemma3:12b model!")
                # Save state to file that model is pulled
                save_state({"gemma3_pulled": True})
                return True
            else:
                print("⚠️ There might have been an issue pulling the model. Check details below:")
                print(pull_result.stdout)
                print(pull_result.stderr)
                save_state({"gemma3_pulled": False})
                return False
        else:
            print("✅ gemma3:12b model is already available")
            # Save state to file that model is pulled
            save_state({"gemma3_pulled": True})
            return True
    except Exception as e:
        print(f"⚠️ Error checking or pulling model: {str(e)}")
        save_state({"gemma3_pulled": False})
        return False


In [ ]:
# Save state to file
def save_state(state_dict):
    try:
        # Read existing state if any
        if os.path.exists(STATE_FILE):
            with open(STATE_FILE, 'r') as f:
                existing_state = json.load(f)
            # Update with new values
            existing_state.update(state_dict)
            state_dict = existing_state

        # Write updated state
        with open(STATE_FILE, 'w') as f:
            json.dump(state_dict, f)
        print(f"State updated: {state_dict}")
    except Exception as e:
        print(f"Error saving state: {str(e)}")

In [ ]:
# Load state from file
def load_state():
    if os.path.exists(STATE_FILE):
        try:
            with open(STATE_FILE, 'r') as f:
                return json.load(f)
        except Exception as e:
            print(f"Error loading state: {str(e)}")
    return {}

In [ ]:
from pdf2image import convert_from_path
import os

def pdf_to_images(pdf_path, output_folder, dpi=300, fmt='jpeg'):
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Convert PDF to images
    images = convert_from_path(pdf_path, dpi=dpi)

    image_paths = []
    for i, image in enumerate(images):
        output_path = os.path.join(output_folder, f'page_{i+1}.{fmt}')
        image.save(output_path, fmt.upper())
        image_paths.append(output_path)
        print(f"Saved: {output_path}")

    return image_paths
# Example usage
pdf_path = '/content/GH_2145172433_18112024.pdf'
output_folder = '/content/converted_image'
pdf_to_images(pdf_path, dpi=300, fmt='jpeg')


TypeError: pdf_to_images() missing 1 required positional argument: 'output_folder'

In [ ]:
# Save Streamlit app code to a file
def save_app_code():
    with open('app.py', 'w') as f:
        f.write('''
import streamlit as st
import ollama
from PIL import Image
import io
import base64
import os
import subprocess
import time
import json
from pdf2image import convert_from_path,convert_from_bytes
import os

# State file to share information between processes
STATE_FILE = '/content/app_state.json'

# Load state from file
def load_state():
    if os.path.exists(STATE_FILE):
        try:
            with open(STATE_FILE, 'r') as f:
                return json.load(f)
        except Exception as e:
            st.sidebar.error(f"Error loading state: {str(e)}")
    return {}

# Save state to file
def save_state(state_dict):
    try:
        # Read existing state if any
        if os.path.exists(STATE_FILE):
            with open(STATE_FILE, 'r') as f:
                existing_state = json.load(f)
            # Update with new values
            existing_state.update(state_dict)
            state_dict = existing_state

        # Write updated state
        with open(STATE_FILE, 'w') as f:
            json.dump(state_dict, f)
    except Exception as e:
        st.sidebar.error(f"Error saving state: {str(e)}")

# Check if gemma3:12b model is available and pull it if not
def check_and_pull_gemma3():
    try:
        # First check if we already pulled it
        state = load_state()
        if state.get('gemma3_pulled', False):
            return True

        # Otherwise check directly
        list_output = subprocess.run("ollama list", shell=True, capture_output=True, text=True)
        if "gemma3:12b" in list_output.stdout:
            save_state({"gemma3_pulled": True})
            return True

        # Need to pull the model
        with st.spinner("📥 Pulling gemma3:12b model (this may take several minutes)..."):
            result = subprocess.run("ollama pull gemma3:12b", shell=True, capture_output=True, text=True)
            if result.returncode == 0:
                save_state({"gemma3_pulled": True})
                st.success("✅ Successfully pulled gemma3:12b model!")
                return True
            else:
                st.error("❌ Failed to pull model. See details below.")
                st.code(result.stderr)
                return False
    except Exception as e:
        st.error(f"Error checking model: {str(e)}")
        return False

# Check if the logo exists, use a fallback if it doesn't
logo_path = "/content/gemma3.png"
if not os.path.exists(logo_path):
    # Create a simple text to display instead
    logo_html = "🔍 "
else:
    # Use the logo image
    with open(logo_path, "rb") as img_file:
        logo_base64 = base64.b64encode(img_file.read()).decode()
        logo_html = f'<img src="data:image/png;base64,{logo_base64}" width="50" style="vertical-align: -12px;">'

# Page configuration
st.set_page_config(
    page_title="Gemma-3 OCR",
    page_icon="🔎",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Title and description in main area
st.markdown(f"""
    # {logo_html} Gemma-3 OCR
""", unsafe_allow_html=True)

# Add clear button to top right
col1, col2 = st.columns([6,1])
with col2:
    if st.button("Clear 🗑️"):
        if 'ocr_result' in st.session_state:
            del st.session_state['ocr_result']
            st.rerun()

st.markdown('<p style="margin-top: -20px;">Extract structured text from images using Gemma-3 Vision!</p>', unsafe_allow_html=True)
st.markdown("---")

# Initialize the app state
if 'model_checked' not in st.session_state:
    st.session_state.model_checked = False

# Check Ollama status on initial load
if 'ollama_running' not in st.session_state:
    status = subprocess.run("pgrep ollama", shell=True, capture_output=True)
    st.session_state.ollama_running = (status.returncode == 0)

# Move upload controls to sidebar
with st.sidebar:
    st.header("Upload Image")
    uploaded_file = st.file_uploader("Choose an image...", type=['png', 'jpg', 'jpeg','pdf'])


    # Check if ollama service is running
    if not st.session_state.ollama_running:
        st.error("❌ Ollama is not running")
        if st.button("Start Ollama Service"):
            subprocess.run("ollama serve > /dev/null 2>&1 &", shell=True)
            time.sleep(3)
            st.session_state.ollama_running = True
            st.success("✅ Started Ollama service")
            st.rerun()
    else:
        st.success("✅ Ollama is running")

    # Check for gemma3 model
    st.subheader("Model Status")

    # Get list of models
    available_models = []
    try:
        if st.session_state.ollama_running:
            # First check if we already know it's pulled
            state = load_state()
            if not state.get('gemma3_pulled', False):
                if st.button("Check Gemma-3 Status"):
                    st.session_state.model_checked = True
                    if check_and_pull_gemma3():
                        st.success("✅ gemma3:12b model is available")
                        st.rerun()
            else:
                st.success("✅ gemma3:12b model is available")

            # Get all available models
            models = ollama.list()
            available_models = [model['name'] for model in models.get('models', [])]
    except Exception as e:
        st.warning(f"Could not fetch models: {str(e)}")

    # Model selection
    if available_models:
        default_index = available_models.index('gemma3:12b') if 'gemma3:12b' in available_models else 0
        selected_model = st.selectbox("Select model:", available_models, index=default_index)
    else:
        state = load_state()
        if state.get('gemma3_pulled', False):
            st.warning("Model list could not be retrieved. Please try restarting the app.")
            selected_model = 'gemma3:12b'
        else:
            selected_model = 'gemma3:12b'  # Default placeholder
            st.warning("No models detected. Check and pull the model first.")

# Main content area
if uploaded_file is not None:
    # Display the uploaded image
    if uploaded_file.type == 'application/pdf':
            # Convert PDF to images (handle first page for display)
            pages = convert_from_bytes(uploaded_file.read(), dpi=200)
            image = pages[0]  # Get the first page as image
            st.image(image, caption="First page of PDF")
    else:
          # Open as image directly
          image = Image.open(uploaded_file)
          st.image(image, caption="Uploaded Image")
    # image = Image.open(uploaded_file)
    # st.image(image, caption="Uploaded Image", use_column_width=True)

    # Make sure the model is available before showing the extract button
    state = load_state()
    if state.get('gemma3_pulled', False) or 'gemma3:12b' in available_models:
        if st.button("Extract Text 🔍", type="primary"):
            with st.spinner("Processing image..."):
                try:
                    response = ollama.chat(
                        model='gemma3:12b',
                        messages=[{
                            'role': 'user',
                            'content': """
                            Analyze the text in the provided image. Extract all readable content
                            and present it in a structured JSON format that is clear, concise,
                            and well-organized. Extract Key-Value Pair information.
                            Make sure all the keys are in camelCase. No other content just JSON Output.
                            """,
                            'images': [uploaded_file.getvalue()]
                        }]
                    )
                    st.session_state['ocr_result'] = response['message']['content']
                except Exception as e:
                    st.error(f"Error processing image: {str(e)}")
                    st.code(str(e), language="python")

                    # Offer to pull model if that might be the issue
                    if "model not found" in str(e).lower() or "no such file" in str(e).lower():
                        st.warning("The model may not be properly installed.")
                        if st.button("Try pulling gemma3:12b again"):
                            check_and_pull_gemma3()
                            st.rerun()
    else:
        st.warning("⚠️ Please verify the gemma3:12b model is available before processing images.")
        if st.button("Check & Pull Model"):
            if check_and_pull_gemma3():
                st.success("✅ Model is now available!")
                st.rerun()

# Main content area for results
if 'ocr_result' in st.session_state:
    st.markdown(st.session_state['ocr_result'])
else:
    st.info("Upload an image and click 'Extract Text' to see the results here.")

# System information
st.sidebar.markdown("---")
st.sidebar.header("System Information")

# Display GPU information if available
st.sidebar.write("GPU Information:")
try:
    gpu_info = subprocess.run("nvidia-smi --query-gpu=name,memory.used,memory.total --format=csv,noheader",
                            shell=True, capture_output=True, text=True)
    if gpu_info.returncode == 0 and gpu_info.stdout.strip():
        st.sidebar.code(gpu_info.stdout)
    else:
        st.sidebar.warning("No GPU detected or nvidia-smi not available")
except:
    st.sidebar.warning("Could not check GPU status")

# Footer
st.markdown("---")
st.markdown("Made with ❤️ using Gemma-3 Vision Model | Running on Google Colab")
''')
    print("Streamlit app code saved to app.py")

In [ ]:
# Function to run Streamlit with ngrok
def run_streamlit_with_ngrok():
    # Set up ngrok
    print("Setting up ngrok tunnel...")
    auth_token = input("Enter your ngrok auth token (get it from https://dashboard.ngrok.com/get-started/your-authtoken): ")
    if auth_token:
        ngrok.set_auth_token(auth_token)

    # Start ngrok tunnel
    public_url = ngrok.connect(8501)
    print(f"\n🚀 Your Streamlit app is LIVE at: {public_url}\n")

    # Run the Streamlit app
    subprocess.Popen(["streamlit", "run", "app.py", "--server.port", "8501"])

    # Keep the notebook running
    print("\nKeep this cell running for the Streamlit app to stay active")
    print("To stop, interrupt the kernel (square button in toolbar)")

    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("Stopping Streamlit and ngrok...")
        ngrok.kill()

# Main execution
ensure_ollama_running()
create_gemma_logo()
model_ready = ensure_gemma3_model()  # Check and pull gemma3:12b model before starting the app
save_app_code()
run_streamlit_with_ngrok()

# Print final status
if model_ready:
    print("\n✅ gemma3:12b model is ready to use")
else:
    print("\n⚠️ There might be issues with the gemma3:12b model")


Starting Ollama service...
Ollama service should be running
Created placeholder logo at /content/gemma3.png
Checking for gemma3:12b model...
✅ gemma3:12b model is already available
State updated: {'gemma3_pulled': True}
Streamlit app code saved to app.py
Setting up ngrok tunnel...

🚀 Your Streamlit app is LIVE at: NgrokTunnel: "https://7ba5-34-139-167-109.ngrok-free.app" -> "http://localhost:8501"


Keep this cell running for the Streamlit app to stay active
To stop, interrupt the kernel (square button in toolbar)
